In [1]:
import tensorflow as tf
import numpy as np
import os
import scipy.io
import random
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
import cv2
from utilities import *

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# cifar10 data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train/255.
x_test = x_test/255.
y_train = np.reshape(y_train,[50000,])
y_test = np.reshape(y_test,[10000,])
nb_classes = 10
targets = y_train.reshape(-1)
y_train = np.eye(nb_classes)[targets]
targets = y_test.reshape(-1)
y_test = np.eye(nb_classes)[targets]

# SVHN data
from scipy import io
data1=io.loadmat('train_32x32.mat')
train_data=data1['X']
train_data = train_data.astype('float32')
train_data = np.transpose(train_data, (3, 0, 1, 2))
SVHN = train_data/255.

# LSUN data
LSUN = np.load("./LSUN.npy")
lSUN = LSUN/255.

# TIM data
TIM = np.load("./TIM.npy")  

batch_size = 128
tf.reset_default_graph()
networks = ['network1', 'network2', 'network3', 'network4', 'network5','network6', 'network7', 'network8', 'network9', 'network10']

XX_list={}
YY_list={}
for mm in range(len(networks)):
  with tf.name_scope(networks[mm]):
    XX_list[networks[mm]] = tf.placeholder(tf.float32, shape = [None, 32, 32, 3],name = networks[mm] +'x')
    YY_list[networks[mm]] = tf.placeholder(tf.float32, [None,10],name =networks[mm] +'y')
    
X = tf.placeholder(tf.float32, shape = [None, 32, 32, 3],name = 'x')
Dcon = tf.placeholder(tf.float32, shape = [None, 32, 32, 3],name = 'Dcon')
Y = tf.placeholder(tf.float32, [None,10],name ='y')
b = tf.placeholder(tf.bool,shape=(),name='b')
learning_rate = tf.placeholder(tf.float32,shape=(),name= 'learning_rate')
alpha = tf.placeholder(tf.float32,shape=(),name='alpha')
stop_grad = tf.placeholder(tf.bool,shape=(),name='stop')

x_list = []
y_list = []
output_list = []
Eent_noise = 0
output1 = 0
pp = 0
for i in range(len(networks)):
    x_image, y_label, output, probs,_,_= vgg16(networks[i],False,XX_list,YY_list,1e-6)
    _, output_noise = vgg16_1(Dcon,networks[i],False,stop_grad,XX_list,YY_list)
    _, _, output_train, probs_train,ent_train,loss= vgg16(networks[i],True,XX_list,YY_list,1e-6)
    x_list.append(x_image)
    y_list.append(y_label)
    output_list.append(output)
    pp+=loss
    Eent_noise += tf.reduce_sum(-tf.log(output_noise+1e-30)*output_noise,1)
    output1 += output_noise
    
output1 = output1/len(networks)
ent_noise = tf.reduce_sum(-tf.log(output1+1e-30)*output1,1)
Eent_noise = Eent_noise/len(networks)
MI = ent_noise - Eent_noise
MI_mean=tf.reduce_mean(MI)
    
pp = pp/len(networks)
loss = pp-MI_mean*alpha
train_opt = tf.train.AdamOptimizer(learning_rate).minimize(loss)

saver = tf.train.Saver(max_to_keep = 10)
config = tf.ConfigProto(allow_soft_placement = True)
sess = tf.Session(config = config)
sess.run(tf.global_variables_initializer())
model_id=0
#save_path = saver.restore(sess, "./density_estimation_cifar10_A/classification_mnist666097/model.ckpt" )
epoch = 391
num_iter = epoch*200
for iter in range(num_iter):

    if iter < 10000:
        a = 0.001
    elif iter < 20000:
        a = 0.0003
    elif iter < 30000:
        a = 0.0001
    elif iter < 40000:
        a = 0.00003
    elif iter < 50000:
        a = 0.00001
    elif iter < 60000:
        a = 0.000003
    elif iter < 70000:
        a = 0.000001
    else:
        a = 0.0000003

    if iter%1000 == 0:
        outputs=[]
        x_test_batch = np.copy(x_test)
        y_test_batch = np.copy(y_test)
        x_test_batch = stand(x_test_batch)
        for i in range(len(networks)):
            probs = []
            for j in range(10):   
              prob= sess.run(output_list[i], {x_list[i]:x_test_batch[j*1000:(j+1)*1000], y_list[i]:y_test_batch[j*1000:(j+1)*1000], b:False})
              probs.extend(prob)  
            outputs.append(probs)
        outputs=np.array(outputs) 
        
        accs = np.mean(np.argmax(outputs,2)==np.argmax(y_test_batch,1),1)
        print("accs: ",accs)

        softmax = np.mean(outputs,0)
        maxp_in = np.max(softmax,1)
        acc = np.mean(np.argmax(softmax,1) ==  np.argmax(y_test_batch,1))
        print( iter, 'Final Testing Accuracy: ', acc)
        
        ent_in = np.sum(-np.log(softmax+1e-11)*softmax,1)
        Eent_in = np.mean(np.sum(-np.log(outputs+1e-11)*outputs,2),0)
        MI_in = ent_in - Eent_in
        
        right = np.argmax(softmax,1) ==  np.argmax(y_test_batch,1)
        wrong_data0 = x_test_batch[~right]
        wrong_data = wrong_data0[np.random.randint(0,len(wrong_data0),10000)]
        right_data = x_test_batch[right]
        right_data = right_data[np.random.randint(0,len(right_data),10000)]
        
        outputs_right=[]
        for i in range(len(networks)):
            prob= sess.run(output_list[i], {x_list[i]:right_data, b:False})
            outputs_right.append(prob)
        outputs_right=np.array(outputs_right) 
        softmax_right = np.mean(outputs_right,0)
        maxp_in_right = np.max(softmax_right,1)
        ent_in_right = np.sum(-np.log(softmax_right+1e-11)*softmax_right,1)
        Eent_in_right = np.mean(np.sum(-np.log(outputs_right+1e-11)*outputs_right,2),0)
        MI_in_right = ent_in_right - Eent_in_right
        
        if iter%7 == 4 or iter%7 == 2 or iter%7 == 6:
            tick=0
            c = 2.
            d = False
        if iter%7 == 5:
            tick=1
            c = 0.8
            d = True
        if iter%7 == 3 or iter%7 == 1 or iter%7 == 0:
            tick=0
            c = 2.
            d = False

        for t in range(0,8):
          if t == 0:
              safe_images = np.reshape(np.random.normal(0,1,[32*32*3*10000]),[10000,32,32,3])
              print("noise:")
          if t == 1:
              safe_images = TIM[np.random.randint(0,10000,10000)]
              print("TIM:")
          if t  == 2:
              safe_images = SVHN[np.random.randint(0,50000,10000)]
              print("SVHN:") 
          if t  == 3:
              safe_images = LSUN
              print("LSUN:")          
          if t == 4:
              safe_images = LSUN
              maxp_in = maxp_in_right
              ent_in = ent_in_right
              MI_in = MI_in_right
              print("LSUN vs true:")  
          if t  == 5:
              safe_images = np.reshape(np.random.normal(0,1,[32*32*3*10000]),[10000,32,32,3])
              maxp_in = maxp_in_right
              ent_in = ent_in_right
              MI_in = MI_in_right
              print("gauss vs true:")
          if t == 6:
              safe_images = TIM[np.random.randint(0,10000,10000)]
              maxp_in = maxp_in_right
              ent_in = ent_in_right
              MI_in = MI_in_right
              print("TIM vs true:")
          if t  == 7:
              safe_images = SVHN[np.random.randint(0,50000,10000)]
              maxp_in = maxp_in_right
              ent_in = ent_in_right
              MI_in = MI_in_right
              print("SVHN vs true:")                    
                
          safe_images = stand(safe_images)   
    
          outputs_OOD=[]
          for j in range(len(networks)):
              probs_OOD  = []
              for r in range(10):
                prob_OOD = sess.run(output_list[j], {x_list[j]:safe_images[r*1000:(r+1)*1000], b:False})
                probs_OOD.extend(prob_OOD)  
              outputs_OOD.append(probs_OOD)
          outputs_OOD = np.array(outputs_OOD) 

          softmax_OOD = np.mean(outputs_OOD,0)
          maxp_OOD = np.max(softmax_OOD,1)
          ent_OOD = np.sum(-np.log(softmax_OOD+1e-11)*softmax_OOD,1)
          Eent_OOD = np.mean(np.sum(-np.log(outputs_OOD+1e-11)*outputs_OOD,2),0)
          MI_OOD = ent_OOD - Eent_OOD

          print("maxp_OOD:",np.mean(maxp_OOD>0.99),np.mean(maxp_OOD),np.std(maxp_OOD))
          print("maxp_inD:",np.mean(maxp_in>0.99),np.mean(maxp_in),np.std(maxp_in))
          print("ent_OOD:",np.mean(ent_OOD),np.std(ent_OOD), "ent_in:", np.mean(ent_in),np.std(ent_in))
          print("MI_OOD:",np.mean(MI_OOD),np.std(MI_OOD), "ent_in:", np.mean(MI_in),np.std(MI_in))

          safe, risky  = -np.reshape(maxp_in,[10000,1]), -np.reshape(maxp_OOD,[10000,1])
          labels = np.zeros((safe.shape[0] + risky.shape[0]), dtype=np.int32)
          labels[safe.shape[0]:] += 1
          examples = np.squeeze(np.vstack((safe, risky)))
          print('AUPR_p:', round(100*average_precision_score(labels, examples), 2))
          print('AUROC_p:', round(100*roc_auc_score(labels, examples), 2))
          print("FPR95:",ErrorRateAt95Recall1(labels, examples)*100)
        
          safe, risky = np.reshape(ent_in,[10000,1]), np.reshape(ent_OOD,[10000,1])
          labels = np.zeros((safe.shape[0] + risky.shape[0]), dtype=np.int32)
          labels[safe.shape[0]:] += 1
          examples = np.squeeze(np.vstack((safe, risky)))
          print('AUPR_entropy:', round(100*average_precision_score(labels, examples), 2))
          print('AUROC_entropy:', round(100*roc_auc_score(labels, examples), 2))
          print("FPR95:",ErrorRateAt95Recall1(labels, examples)*100)
        
          safe, risky = np.reshape(MI_in,[10000,1]), np.reshape(MI_OOD,[10000,1])
          labels = np.zeros((safe.shape[0] + risky.shape[0]), dtype=np.int32)
          labels[safe.shape[0]:] += 1
          examples = np.squeeze(np.vstack((safe, risky)))
          print('AUPR_MI:', round(100*average_precision_score(labels, examples), 2))
          print('AUROC_MI:', round(100*roc_auc_score(labels, examples), 2))
          print("FPR95:",ErrorRateAt95Recall1(labels, examples)*100)
          print("############################################")
        
    if tick == 0 and iter%1==0:
        my_noise = np.copy(x_train[np.random.randint(0,50000,batch_size)])
        my_noise = crop(my_noise)
        my_noise = stand(my_noise)
    if tick == 1 and iter%100==0:
        my_noise = np.copy(x_train[np.random.randint(0,50000,batch_size)])
        my_noise = preprocess(my_noise)
        
    x_batch_list = []
    y_batch_list = []
    for hh in range(len(networks)):
        seed = np.random.randint(0,50000,batch_size)
        x_batch = x_train[seed]
        y_batch = y_train[seed]
        x_batch = preprocess(x_batch) 
        x_batch_list.append(x_batch)
        y_batch_list.append(y_batch)     
    
    sess.run(train_opt, {x_list[0]:x_batch_list[0], y_list[0]:y_batch_list[0],
                         x_list[1]:x_batch_list[1], y_list[1]:y_batch_list[1],
                         x_list[2]:x_batch_list[2], y_list[2]:y_batch_list[2],
                         x_list[3]:x_batch_list[3], y_list[3]:y_batch_list[3],
                         x_list[4]:x_batch_list[4], y_list[4]:y_batch_list[4],
                         x_list[5]:x_batch_list[5], y_list[5]:y_batch_list[5],
                         x_list[6]:x_batch_list[6], y_list[6]:y_batch_list[6],
                         x_list[7]:x_batch_list[7], y_list[7]:y_batch_list[7],
                         x_list[8]:x_batch_list[8], y_list[8]:y_batch_list[8],
                         x_list[9]:x_batch_list[9], y_list[9]:y_batch_list[9], 
                         alpha:c, b:True, Dcon:my_noise, learning_rate:a,stop_grad:d})  




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.


accs:  [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
0 Final Testing Accuracy:  0.1
noise:
maxp_OOD: 0.0 0.143953 4.817199e-06
maxp_inD: 0.0 0.14393634 8.820219e-06
ent_OOD: 2.287818 2.2327883e-06 ent_in: 2.287829 5.213513e-06
MI_OOD: 0.066102766 4.3707873e-06 ent_in: 0.06611416 8.80399e-06
AUPR_p: 31.51
AUROC_p: 5.57
FPR95: 99.05000000000001
AUPR_entropy: 31.88
AUROC_entropy: 3.

maxp_OOD: 0.0 0.26071203 0.077845484
maxp_inD: 0.0 0.31955394 0.12799531
ent_OOD: 1.9160914 0.1678637 ent_in: 1.8683004 0.2683916
MI_OOD: 1.455741 0.27158067 ent_in: 1.552054 0.33300424
AUPR_p: 58.34
AUROC_p: 62.3
FPR95: 76.52
AUPR_entropy: 49.07
AUROC_entropy: 51.54
FPR95: 80.39
AUPR_MI: 41.68
AUROC_MI: 40.32
FPR95: 94.8
############################################
accs:  [0.4373 0.4277 0.3434 0.6314 0.2806 0.604  0.5797 0.4268 0.432  0.192 ]
2000 Final Testing Accuracy:  0.7384
noise:
maxp_OOD: 0.0 0.22206731 0.046102043
maxp_inD: 0.0007 0.447867 0.21153902
ent_OOD: 2.1251698 0.05658173 ent_in: 1.5424563 0.5033455
MI_OOD: 1.9689568 0.08614321 ent_in: 1.0852833 0.44676134
AUPR_p: 78.59
AUROC_p: 86.49
FPR95: 28.02
AUPR_entropy: 82.12
AUROC_entropy: 91.81
FPR95: 15.120000000000001
AUPR_MI: 91.49
AUROC_MI: 96.77
FPR95: 5.9799999999999995
############################################
TIM:
maxp_OOD: 0.0 0.20414102 0.10172854
maxp_inD: 0.0007 0.447867 0.21153902
ent_OOD: 2.1043398 0.22967441

maxp_OOD: 0.0 0.23687443 0.076318115
maxp_inD: 0.0146 0.5925046 0.20831054
ent_OOD: 2.0064368 0.15237789 ent_in: 1.227048 0.5318734
MI_OOD: 1.4294707 0.30950826 ent_in: 0.89336896 0.45280346
AUPR_p: 93.56
AUROC_p: 95.1
FPR95: 17.849999999999998
AUPR_entropy: 90.69
AUROC_entropy: 93.8
FPR95: 17.39
AUPR_MI: 78.43
AUROC_MI: 83.16
FPR95: 42.480000000000004
############################################
gauss vs true:
maxp_OOD: 0.0 0.24831931 0.0426096
maxp_inD: 0.0146 0.5925046 0.20831054
ent_OOD: 1.9792582 0.0624655 ent_in: 1.227048 0.5318734
MI_OOD: 1.6284118 0.13205643 ent_in: 0.89336896 0.45280346
AUPR_p: 90.17
AUROC_p: 95.5
FPR95: 11.99
AUPR_entropy: 85.64
AUROC_entropy: 94.18
FPR95: 11.44
AUPR_MI: 86.92
AUROC_MI: 93.84
FPR95: 14.299999999999999
############################################
TIM vs true:
maxp_OOD: 0.0 0.23504949 0.11744933
maxp_inD: 0.0146 0.5925046 0.20831054
ent_OOD: 2.0370603 0.2528966 ent_in: 1.227048 0.5318734
MI_OOD: 1.6759902 0.4092914 ent_in: 0.89336896 0.45280346

noise:
maxp_OOD: 0.0 0.18266773 0.040375065
maxp_inD: 0.0982 0.7151092 0.22197615
ent_OOD: 2.1455655 0.063347176 ent_in: 0.8632341 0.585397
MI_OOD: 1.8754039 0.14439327 ent_in: 0.54864377 0.41597304
AUPR_p: 99.25
AUROC_p: 99.47
FPR95: 2.52
AUPR_entropy: 98.71
AUROC_entropy: 99.47
FPR95: 1.49
AUPR_MI: 99.22
AUROC_MI: 99.64
FPR95: 1.15
############################################
TIM:
maxp_OOD: 0.0001 0.26342106 0.1378106
maxp_inD: 0.0982 0.7151092 0.22197615
ent_OOD: 1.9884856 0.3042139 ent_in: 0.8632341 0.585397
MI_OOD: 1.5963444 0.46083257 ent_in: 0.54864377 0.41597304
AUPR_p: 94.7
AUROC_p: 94.74
FPR95: 24.279999999999998
AUPR_entropy: 95.18
AUROC_entropy: 95.25
FPR95: 22.36
AUPR_MI: 94.61
AUROC_MI: 94.52
FPR95: 27.35
############################################
SVHN:
maxp_OOD: 0.0001 0.48800957 0.16009727
maxp_inD: 0.0982 0.7151092 0.22197615
ent_OOD: 1.3791069 0.3355647 ent_in: 0.8632341 0.585397
MI_OOD: 0.5759384 0.21463872 ent_in: 0.54864377 0.41597304
AUPR_p: 70.68
AUROC_p: 78.59

gauss vs true:
maxp_OOD: 0.0 0.22014695 0.043313917
maxp_inD: 0.0219 0.68812853 0.18459111
ent_OOD: 2.1193476 0.07908718 ent_in: 0.9711339 0.48513886
MI_OOD: 1.9639809 0.1567619 ent_in: 0.71348554 0.38299808
AUPR_p: 98.98
AUROC_p: 99.29
FPR95: 2.87
AUPR_entropy: 99.4
AUROC_entropy: 99.62
FPR95: 1.6500000000000001
AUPR_MI: 99.75
AUROC_MI: 99.78
FPR95: 1.01
############################################
TIM vs true:
maxp_OOD: 0.0 0.24702929 0.1272269
maxp_inD: 0.0219 0.68812853 0.18459111
ent_OOD: 2.0239449 0.27023116 ent_in: 0.9711339 0.48513886
MI_OOD: 1.6733979 0.42413065 ent_in: 0.71348554 0.38299808
AUPR_p: 96.68
AUROC_p: 96.51
FPR95: 18.279999999999998
AUPR_entropy: 96.91
AUROC_entropy: 96.77
FPR95: 17.45
AUPR_MI: 95.02
AUROC_MI: 94.68
FPR95: 30.080000000000002
############################################
SVHN vs true:
maxp_OOD: 0.0 0.43003935 0.12036431
maxp_inD: 0.0219 0.68812853 0.18459111
ent_OOD: 1.4696988 0.25739506 ent_in: 0.9711339 0.48513886
MI_OOD: 0.7592041 0.19459607 ent_

...

maxp_OOD: 0.0 0.31610397 0.09076555
maxp_inD: 0.3593 0.87195975 0.17290537
ent_OOD: 1.8238782 0.18901618 ent_in: 0.3876504 0.43875265
MI_OOD: 1.0880584 0.19888122 ent_in: 0.20562014 0.25529596
AUPR_p: 98.53
AUROC_p: 98.78
FPR95: 5.46
AUPR_entropy: 99.2
AUROC_entropy: 99.38
FPR95: 2.3
AUPR_MI: 98.43
AUROC_MI: 98.92
FPR95: 4.04
############################################
LSUN vs true:
maxp_OOD: 0.0 0.31610397 0.09076555
maxp_inD: 0.3918 0.8959412 0.14945082
ent_OOD: 1.8238782 0.18901618 ent_in: 0.33132344 0.39470622
MI_OOD: 1.0880584 0.19888122 ent_in: 0.17680766 0.23596638
AUPR_p: 99.24
AUROC_p: 99.34
FPR95: 3.1300000000000003
AUPR_entropy: 99.53
AUROC_entropy: 99.62
FPR95: 1.3
AUPR_MI: 98.54
AUROC_MI: 99.14
FPR95: 2.85
############################################
gauss vs true:
maxp_OOD: 0.0 0.38149598 0.03338518
maxp_inD: 0.3918 0.8959412 0.14945082
ent_OOD: 1.6283004 0.07360587 ent_in: 0.33132344 0.39470622
MI_OOD: 1.4165215 0.085950844 ent_in: 0.17680766 0.23596638
AUPR_p: 97.59
AU

accs:  [0.8782 0.8746 0.8613 0.8377 0.868  0.9071 0.8989 0.8507 0.9005 0.86  ]
71000 Final Testing Accuracy:  0.9275
noise:
maxp_OOD: 0.0 0.38084716 0.033227205
maxp_inD: 0.4225 0.8846833 0.16868694
ent_OOD: 1.632802 0.06871556 ent_in: 0.34065533 0.42196304
MI_OOD: 1.4212039 0.080908746 ent_in: 0.17901829 0.24218914
AUPR_p: 95.9
AUROC_p: 98.44
FPR95: 2.22
AUPR_entropy: 97.22
AUROC_entropy: 99.15
FPR95: 1.52
AUPR_MI: 99.89
AUROC_MI: 99.96
FPR95: 0.06999999999999999
############################################
TIM:
maxp_OOD: 0.0034 0.45162255 0.18171054
maxp_inD: 0.4225 0.8846833 0.16868694
ent_OOD: 1.4834266 0.42344457 ent_in: 0.34065533 0.42196304
MI_OOD: 0.926242 0.37565365 ent_in: 0.17901829 0.24218914
AUPR_p: 93.37
AUROC_p: 94.04
FPR95: 22.37
AUPR_entropy: 95.06
AUROC_entropy: 95.42
FPR95: 19.74
AUPR_MI: 95.05
AUROC_MI: 95.27
FPR95: 21.029999999999998
############################################
SVHN:
maxp_OOD: 0.0004 0.48655048 0.1506784
maxp_inD: 0.4225 0.8846833 0.16868694
ent_OO

maxp_OOD: 0.0 0.31056044 0.089166805
maxp_inD: 0.1963 0.8422507 0.15699634
ent_OOD: 1.8282945 0.19034702 ent_in: 0.51372063 0.4312094
MI_OOD: 1.148772 0.21500017 ent_in: 0.34793946 0.31725904
AUPR_p: 98.99
AUROC_p: 99.12
FPR95: 3.81
AUPR_entropy: 99.23
AUROC_entropy: 99.35
FPR95: 2.7
AUPR_MI: 95.98
AUROC_MI: 97.04
FPR95: 10.040000000000001
############################################
gauss vs true:
maxp_OOD: 0.0 0.33306733 0.037034016
maxp_inD: 0.1963 0.8422507 0.15699634
ent_OOD: 1.6375946 0.06354268 ent_in: 0.51372063 0.4312094
MI_OOD: 1.4484153 0.08320456 ent_in: 0.34793946 0.31725904
AUPR_p: 98.18
AUROC_p: 99.34
FPR95: 1.71
AUPR_entropy: 95.89
AUROC_entropy: 98.74
FPR95: 2.09
AUPR_MI: 99.66
AUROC_MI: 99.84
FPR95: 0.42
############################################
TIM vs true:
maxp_OOD: 0.0007 0.4052292 0.15699378
maxp_inD: 0.1963 0.8422507 0.15699634
ent_OOD: 1.6014208 0.35572407 ent_in: 0.51372063 0.4312094
MI_OOD: 1.1255647 0.33530903 ent_in: 0.34793946 0.31725904
AUPR_p: 95.59
AU

noise:
maxp_OOD: 0.0 0.38120905 0.03072121
maxp_inD: 0.4 0.88169104 0.16911103
ent_OOD: 1.6329074 0.06902916 ent_in: 0.3521915 0.42349365
MI_OOD: 1.4252919 0.079461865 ent_in: 0.18554065 0.24324699
AUPR_p: 95.49
AUROC_p: 98.39
FPR95: 2.17
AUPR_entropy: 97.19
AUROC_entropy: 99.14
FPR95: 1.51
AUPR_MI: 99.86
AUROC_MI: 99.96
FPR95: 0.06999999999999999
############################################
TIM:
maxp_OOD: 0.0029 0.45269254 0.18073288
maxp_inD: 0.4 0.88169104 0.16911103
ent_OOD: 1.4809505 0.41927674 ent_in: 0.3521915 0.42349365
MI_OOD: 0.91838914 0.371303 ent_in: 0.18554065 0.24324699
AUPR_p: 93.2
AUROC_p: 93.9
FPR95: 22.53
AUPR_entropy: 94.93
AUROC_entropy: 95.29
FPR95: 20.330000000000002
AUPR_MI: 94.83
AUROC_MI: 95.06
FPR95: 21.72
############################################
SVHN:
maxp_OOD: 0.0001 0.4799605 0.14750083
maxp_inD: 0.4 0.88169104 0.16911103
ent_OOD: 1.3523384 0.3144572 ent_in: 0.3521915 0.42349365
MI_OOD: 0.9089581 0.29298773 ent_in: 0.18554065 0.24324699
AUPR_p: 91.78
A

gauss vs true:
maxp_OOD: 0.0 0.3802334 0.030335251
maxp_inD: 0.4528 0.9086715 0.14374475
ent_OOD: 1.6365274 0.06807527 ent_in: 0.28503057 0.37353605
MI_OOD: 1.436238 0.07504926 ent_in: 0.15074174 0.21820337
AUPR_p: 97.63
AUROC_p: 99.24
FPR95: 1.11
AUPR_entropy: 98.43
AUROC_entropy: 99.54
FPR95: 0.86
AUPR_MI: 99.92
AUROC_MI: 99.97
FPR95: 0.04
############################################
TIM vs true:
maxp_OOD: 0.0041 0.4561494 0.18334886
maxp_inD: 0.4528 0.9086715 0.14374475
ent_OOD: 1.4712908 0.42655522 ent_in: 0.28503057 0.37353605
MI_OOD: 0.913437 0.3717457 ent_in: 0.15074174 0.21820337
AUPR_p: 95.31
AUROC_p: 95.68
FPR95: 18.14
AUPR_entropy: 96.35
AUROC_entropy: 96.53
FPR95: 16.25
AUPR_MI: 96.03
AUROC_MI: 96.19
FPR95: 17.51
############################################
SVHN vs true:
maxp_OOD: 0.0003 0.48324326 0.14769568
maxp_inD: 0.4528 0.9086715 0.14374475
ent_OOD: 1.3416224 0.31685716 ent_in: 0.28503057 0.37353605
MI_OOD: 0.8990974 0.29575327 ent_in: 0.15074174 0.21820337
AUPR_p: 94